# Image Captioning Project

## Training the network

---
In this notebook, we adjust our architectures and hyperparameters while training on the COCO dataset.

<a id='step1'></a>
## Step 1: Training Setup

Important parameters that effects the training of our network:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append(r'C:\Users\Project\Documents\Python Scripts\Github_Clone\cocoapi-master\PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

batch_size = 64          # batch size
vocab_threshold = 3        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 1048          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 200          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.0001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)
total_step

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.92s)
creating index...
index created!
Obtaining caption lengths...


100%|████████████████████████████████████████████████████████████████████████| 414113/414113 [00:56<00:00, 7316.64it/s]
C:\Users\Project\Anaconda3\envs\Sylar_CV\lib\site-packages\torch\nn\modules\rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


6471

<a id='step2'></a>
## Step 2: Train our Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

In [2]:
import torch.utils.data as data
import numpy as np
import os
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()


for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
      
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [1/6471], Loss: 9.3614, Perplexity: 11630.2678the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [1/3], Step [2/6471], Loss: 9.3488, Perplexity: 11484.7450the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [3/6471], Loss: 9.3309, Perplexity: 11280.7443the hiddens.shape: torch.Size([64, 11, 1048])
Epoch [1/3], Step [4/6471], Loss: 9.3269, Perplexity: 11235.8326the hiddens.shape: torch.Size([64, 26, 1048])
Epoch [1/3], Step [5/6471], Loss: 9.3194, Perplexity: 11152.6376the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [1/3], Step [6/6471], Loss: 9.2895, Perplexity: 10824.0624the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [7/6471], Loss: 9.2787, Perplexity: 10707.4609the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [8/6471], Loss: 9.2581, Perplexity: 10488.8602the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [9/6471], Loss: 9.2511, Perplexity: 10415.9531the hidden

Epoch [1/3], Step [75/6471], Loss: 5.3452, Perplexity: 209.6003the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [76/6471], Loss: 5.4562, Perplexity: 234.2149the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [77/6471], Loss: 5.3846, Perplexity: 218.0242the hiddens.shape: torch.Size([64, 11, 1048])
Epoch [1/3], Step [78/6471], Loss: 5.6499, Perplexity: 284.2695the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [79/6471], Loss: 5.2254, Perplexity: 185.9341the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [1/3], Step [80/6471], Loss: 5.3495, Perplexity: 210.5120the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [81/6471], Loss: 5.3541, Perplexity: 211.4757the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [82/6471], Loss: 5.3385, Perplexity: 208.1934the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [83/6471], Loss: 5.3123, Perplexity: 202.8217the hiddens.shape: torch.Size([64, 18, 1048])
Epoch [1/3], Step [

Epoch [1/3], Step [149/6471], Loss: 4.9074, Perplexity: 135.2891the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [150/6471], Loss: 4.8782, Perplexity: 131.3898the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [1/3], Step [151/6471], Loss: 4.9639, Perplexity: 143.1479the hiddens.shape: torch.Size([64, 25, 1048])
Epoch [1/3], Step [152/6471], Loss: 5.4775, Perplexity: 239.2377the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [1/3], Step [153/6471], Loss: 5.1129, Perplexity: 166.1477the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [154/6471], Loss: 4.8338, Perplexity: 125.6881the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [1/3], Step [155/6471], Loss: 5.1011, Perplexity: 164.1962the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [1/3], Step [156/6471], Loss: 4.9338, Perplexity: 138.9102the hiddens.shape: torch.Size([64, 17, 1048])
Epoch [1/3], Step [157/6471], Loss: 4.9932, Perplexity: 147.4043the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3

Epoch [1/3], Step [223/6471], Loss: 5.0476, Perplexity: 155.6443the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [224/6471], Loss: 4.7047, Perplexity: 110.4676the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [1/3], Step [225/6471], Loss: 4.7121, Perplexity: 111.2840the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [226/6471], Loss: 4.6455, Perplexity: 104.1195the hiddens.shape: torch.Size([64, 11, 1048])
Epoch [1/3], Step [227/6471], Loss: 4.9383, Perplexity: 139.5323the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [1/3], Step [228/6471], Loss: 4.6734, Perplexity: 107.0624the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [1/3], Step [229/6471], Loss: 4.6371, Perplexity: 103.2470the hiddens.shape: torch.Size([64, 22, 1048])
Epoch [1/3], Step [230/6471], Loss: 5.4075, Perplexity: 223.0639the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [231/6471], Loss: 4.6187, Perplexity: 101.3625the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [1/3

Epoch [1/3], Step [297/6471], Loss: 4.3830, Perplexity: 80.0808the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [298/6471], Loss: 4.3632, Perplexity: 78.5045the hiddens.shape: torch.Size([64, 16, 1048])
Epoch [1/3], Step [299/6471], Loss: 4.5877, Perplexity: 98.2670the hiddens.shape: torch.Size([64, 17, 1048])
Epoch [1/3], Step [300/6471], Loss: 4.4213, Perplexity: 83.2069the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [301/6471], Loss: 4.2205, Perplexity: 68.0662the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [302/6471], Loss: 4.2548, Perplexity: 70.4409the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [1/3], Step [303/6471], Loss: 4.4040, Perplexity: 81.7781the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [1/3], Step [304/6471], Loss: 4.3619, Perplexity: 78.4073the hiddens.shape: torch.Size([64, 11, 1048])
Epoch [1/3], Step [305/6471], Loss: 4.7269, Perplexity: 112.9504the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [1/3], Step 

...

Epoch [3/3], Step [6161/6471], Loss: 2.0202, Perplexity: 7.5400the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6162/6471], Loss: 2.1203, Perplexity: 8.3333the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step [6163/6471], Loss: 2.1871, Perplexity: 8.9093the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [3/3], Step [6164/6471], Loss: 2.1016, Perplexity: 8.1792the hiddens.shape: torch.Size([64, 17, 1048])
Epoch [3/3], Step [6165/6471], Loss: 2.3213, Perplexity: 10.1893the hiddens.shape: torch.Size([64, 16, 1048])
Epoch [3/3], Step [6166/6471], Loss: 2.2235, Perplexity: 9.2394the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step [6167/6471], Loss: 2.2713, Perplexity: 9.6921the hiddens.shape: torch.Size([64, 11, 1048])
Epoch [3/3], Step [6168/6471], Loss: 2.1543, Perplexity: 8.6218the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [3/3], Step [6169/6471], Loss: 2.0674, Perplexity: 7.9040the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step 

Epoch [3/3], Step [6236/6471], Loss: 2.1480, Perplexity: 8.5681the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step [6237/6471], Loss: 2.1318, Perplexity: 8.4304the hiddens.shape: torch.Size([64, 16, 1048])
Epoch [3/3], Step [6238/6471], Loss: 2.0921, Perplexity: 8.1016the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6239/6471], Loss: 1.9944, Perplexity: 7.3475the hiddens.shape: torch.Size([64, 17, 1048])
Epoch [3/3], Step [6240/6471], Loss: 2.2334, Perplexity: 9.3319the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [3/3], Step [6241/6471], Loss: 2.2443, Perplexity: 9.4339the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6242/6471], Loss: 2.0869, Perplexity: 8.0599the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6243/6471], Loss: 1.9504, Perplexity: 7.0316the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6244/6471], Loss: 1.9486, Perplexity: 7.0186the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step [

Epoch [3/3], Step [6311/6471], Loss: 1.8359, Perplexity: 6.2708the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6312/6471], Loss: 2.1156, Perplexity: 8.2946the hiddens.shape: torch.Size([64, 17, 1048])
Epoch [3/3], Step [6313/6471], Loss: 2.3705, Perplexity: 10.7023the hiddens.shape: torch.Size([64, 20, 1048])
Epoch [3/3], Step [6314/6471], Loss: 2.6081, Perplexity: 13.5728the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [3/3], Step [6315/6471], Loss: 1.9820, Perplexity: 7.2571the hiddens.shape: torch.Size([64, 15, 1048])
Epoch [3/3], Step [6316/6471], Loss: 2.1363, Perplexity: 8.4684the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step [6317/6471], Loss: 2.2264, Perplexity: 9.2662the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6318/6471], Loss: 2.1689, Perplexity: 8.7489the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6319/6471], Loss: 1.8974, Perplexity: 6.6685the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step

Epoch [3/3], Step [6386/6471], Loss: 2.3351, Perplexity: 10.3306the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6387/6471], Loss: 2.0527, Perplexity: 7.7885the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6388/6471], Loss: 1.9637, Perplexity: 7.1254the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6389/6471], Loss: 2.1255, Perplexity: 8.3775the hiddens.shape: torch.Size([64, 11, 1048])
Epoch [3/3], Step [6390/6471], Loss: 2.4953, Perplexity: 12.1257the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6391/6471], Loss: 2.1672, Perplexity: 8.7336the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6392/6471], Loss: 1.9336, Perplexity: 6.9146the hiddens.shape: torch.Size([64, 16, 1048])
Epoch [3/3], Step [6393/6471], Loss: 2.3016, Perplexity: 9.9902the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6394/6471], Loss: 2.1237, Perplexity: 8.3620the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step

Epoch [3/3], Step [6461/6471], Loss: 2.2944, Perplexity: 9.9180the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step [6462/6471], Loss: 2.0846, Perplexity: 8.0415the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6463/6471], Loss: 1.9218, Perplexity: 6.8334the hiddens.shape: torch.Size([64, 12, 1048])
Epoch [3/3], Step [6464/6471], Loss: 2.0234, Perplexity: 7.5641the hiddens.shape: torch.Size([64, 17, 1048])
Epoch [3/3], Step [6465/6471], Loss: 2.3343, Perplexity: 10.3221the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6466/6471], Loss: 1.9337, Perplexity: 6.9151the hiddens.shape: torch.Size([64, 13, 1048])
Epoch [3/3], Step [6467/6471], Loss: 1.8836, Perplexity: 6.5773the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step [6468/6471], Loss: 1.9484, Perplexity: 7.0176the hiddens.shape: torch.Size([64, 16, 1048])
Epoch [3/3], Step [6469/6471], Loss: 2.1847, Perplexity: 8.8880the hiddens.shape: torch.Size([64, 14, 1048])
Epoch [3/3], Step 